# Training deep neural nets
Adapted from Chap. 11 of `Hands-on Machine Learning with Scikit-Learn and TensorFlow` by A. Geron.

### Main problems in training deep neural networks with millions of parametetrs
1. Vanishing (or exploding) gradients that make lower layers hard to train
2. Training is very slow
3. Severe overfitting possible

## Vanishing gradients problem
Backpropagation algorithm works by going from the output layer to the input layer, propagating the error gradient along the way. Gradients often get smaller and smaller as the algorithm progresses down to lower (closer to input) layers. Therefore, gradient descent (GD) update leaves the connection weights in the lower layers virtually unchanged and optimization never converges. Neural networks may also suffer from the _exploding gradients_ problem. This is especially the case with recurrent neural networks.

In 2010, Glorot and Bengio showed that with logistic activation function and a simple mean-zero-std-one initialization, the variances of the outputs of each layer are much larger than the variances of the inputs. Therefore, the outputs in the top layers saturate close to zero or one and the gradients are very small. Therefore, the backpropagation algorithm has basically no gradient to propagate to the lower layers.

As a solution, Glorot and Bengio proposed that the connection weights be initialized with zero-mean normal distribution with the following standard deviation $\sigma$ or uniform distribution with the range $-r$ and $+r$:

$$
    \sigma = \sqrt{\frac{2}{n_{inputs} + n_{outputs}}}, \\
    r = \sqrt{\frac{6}{n_{inputs} + n_{outputs}}}
$$
Here $n_{inputs}$ and $n_{outputs}$ are the number of input and output connections for each layer. By default, `tf.layers.dense()` uses this initialization with a uniform distribution. One can use the similar _He initialization_ as follows:
```python
    he_init = tf.contrib.layers.variance_scaling_initializer(mode='FAN_AVG')
    hidden1 = tf.layers.dense(X, n_hidden1, activation=tf.nn.relu, kernel_initializer=he_init, name="hidden1")
```

### Nonsaturating activation functions
As mentioned above, using the logistic activation function can lead to the vanishing gradients problem. For example, ReLU works well because it does not saturate for large positive values and it also fast to compute. ReLU suffers, however, partially from the same problem as the logistic activation function that the output values can saturate to zero. To solve this problem, one can use a variant of the ReLU such as `LeakyReLU`$(z)=\max(\alpha z, z)$, where typically $\alpha=0.01$. The non-zero $alpha$ ensures that leaky ReLUs never completely die.

Another variant is the exponential linear unit (ELU) defined as

$$
\textrm{ELU}(z) = \left\{\begin{array}{lr}
        \alpha(e^z - 1), & \text{for } z < 0 \\
        z, & \text{for } z \geq 0
        \end{array}\right.
$$

Compared to ReLU, ELU has nonzero gradient for $z < 0$, is smooth everywhere, and has an average output closer to 0 at $x=0$. All these features mitigate the vanishing gradients problem. The only disadvantage with ELU is that it is slower to compute than ReLU.

### Batch normalization
He initialization and variants of ELU reduce the vanishing gradients problem at the beginning of training, but nothing guarantees it does not re-emerge during training. In 2015, Ioffe and Szegedy [proposed](https://arxiv.org/pdf/1502.03167v3.pdf) a technique called _Batch normalization_ (BN) to address the vanishing/exploding gradients problems, or more generally the problem that the distribution of each layer's inputs changes during training, as the parameters of the previous layers change (Internal covariance shift problem). 

The technique consists of adding an operation in the model just before the activation function of each layer: zero-centering and normalizing the inputs, then scaling and shifting the result using two new parameters per layer (one for scaling, the other for shifting). This operationg lets the model learn the optimal scale and inputs for each layer.

The algorithm estimates the inputs' mean and standard deviation by evaluating the mean and standard deviation of the inputs over the current mini-batch (hence the name Batch Normalization). In total, four parameters are learned for each batch-normalized layer: $\gamma$ (scale of outputs), $\beta$ (offset of outputs), $\mu$ (mean of inputs), and $\sigma$ (standard deviation of inputs).

Ioffe and Szegedy showed that using batch normalization strongly reduced the vanishing gradients problem, reduced the sensitivity of training to the weight initialization, allowed for using much larger learning rates, and even acted as a regularization mitigating overfitting. Batch normalization adds complexity to the model and slows down training due to the extra computations required. Training can speed up once GD has found reasonably good values for the scales and offsets, though.

See the example below for batch normalization in TensorFlow.

### Gradient clipping
One technique to mitigate the exploding gradients problem is to [clip the gradients](http://proceedings.mlr.press/v28/pascanu13.pdf) during backpropagation so that they never exceed a given threshold. In TensorFlow, the optimizer's `minimize` function both computes the gradients and applies them to variables, so one must instead compute the gradients first, then create an operation to clip the gradients by value and finally apply the clipped gradients. The clip threshold is a hyperparameter that can be tuned. See the example below for gradient clipping.

### Full example from Chap. 10 with He initialization, ELU activation function, batch normalization, and gradient clipping

In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
from functools import partial
import os
from datetime import datetime

n_inputs = 28*28
n_hidden1 = 300
n_hidden2 = 100
n_outputs = 10

tf.reset_default_graph()

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X") # Number of training samples in batch not known and not required
y = tf.placeholder(tf.int64, shape=(None), name="y") # y is a 1D array with unknown length

# This will be set to True during training to tell batch normalization layers to use the whole training set's mean and stddev
training = tf.placeholder_with_default(False, shape=(), name='training')

he_init = tf.contrib.layers.variance_scaling_initializer(mode='FAN_AVG')

# BN uses exponential decay with momentum to compute the running averages
batch_norm_layer = partial(tf.layers.batch_normalization, training=training, momentum=0.9)

# Neural network with batch-normalized layers
with tf.name_scope("ann"):
    hidden1 = tf.layers.dense(X, n_hidden1, name="hidden1", kernel_initializer=he_init)
    bn1 = batch_norm_layer(hidden1)
    bn1_act = tf.nn.elu(bn1)
    hidden2 = tf.layers.dense(bn1_act, n_hidden2, name="hidden2", kernel_initializer=he_init)
    bn2 = batch_norm_layer(hidden2)
    bn2_act = tf.nn.elu(bn2)
    logits_before_bn = tf.layers.dense(bn2_act, n_outputs, name="outputs", kernel_initializer=he_init) # Unscaled as softmax computed later internally
    logits = batch_norm_layer(logits_before_bn, training=training, momentum=0.9)
    

with tf.name_scope("loss"):
    # This op expects unscaled logits, since it performs a softmax on logits internally for efficiency.
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    loss = tf.reduce_mean(xentropy, name="loss")
    
learning_rate = 0.05
gradient_clip_threshold = 1.0

# Training with gradient clipping
with tf.name_scope("train"):
    global_step = tf.Variable(0, name='global_step', trainable=False)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
    grads_and_vars = optimizer.compute_gradients(loss)
    capped_cvs = [(tf.clip_by_value(grad, -gradient_clip_threshold, gradient_clip_threshold), var) 
                  for grad, var in grads_and_vars]
    training_op = optimizer.apply_gradients(capped_cvs, global_step=global_step)

# Evaluate performance by checking if the correct label is in top 1:
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    with tf.name_scope("accuracy"):
        accuracy = tf.reduce_mean(tf.cast(correct, tf.float32), name='accuracy')

tf.summary.scalar('accuracy', accuracy)
summaries = tf.summary.merge_all()
   
saver = tf.train.Saver()

mnist = input_data.read_data_sets("/tmp/mnist/data")

n_epochs = 10
batch_size = 50

# Batch normalization creates operations that must be evaluated at each step to update the moving averages
extra_update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)

root_logdir = 'mnist-logs'

def tb_logdir():   
    now = datetime.now().strftime("%Y-%m-%dT%H:%M:%S")
    return os.path.join(root_logdir, 'run-%s' % now)

logdir = tb_logdir()
print('Using %s for TensorBoard logs' % logdir)

SAVED_MODEL_PATH = os.path.join(logdir, 'model.ckpt')

'''
# This could be useful later
def variable_summaries(var):
    """Attach a lot of summaries to a Tensor (for TensorBoard visualization)."""
    with tf.name_scope('summaries'):
        mean = tf.reduce_mean(var)
        tf.summary.scalar('mean', mean)
        with tf.name_scope('stddev'):
            stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))
            tf.summary.scalar('stddev', stddev)
        tf.summary.scalar('max', tf.reduce_max(var))
        tf.summary.scalar('min', tf.reduce_min(var))
        tf.summary.histogram('histogram', var)
'''
        
with tf.Session() as sess:
    file_writer = tf.summary.FileWriter(logdir, sess.graph)
    tf.global_variables_initializer().run()
    for epoch in range(n_epochs):
        for iteration in range(mnist.train.num_examples // batch_size):
            X_batch, y_batch = mnist.train.next_batch(batch_size)
            summary, _, _ = sess.run([summaries, training_op, extra_update_ops], feed_dict={X: X_batch, y: y_batch})
        acc_train = accuracy.eval(feed_dict={training: True, X: X_batch, y: y_batch})
        acc_val = accuracy.eval(feed_dict={X: mnist.validation.images, y: mnist.validation.labels})
        print(epoch, "Train accuracy:", acc_train, "Val accuracy", acc_val)
        file_writer.add_summary(summary, epoch)
    save_path = saver.save(sess, SAVED_MODEL_PATH)
    file_writer.close()

/Users/ksaaskil/.pyenv/versions/3.5.0/envs/python3.5.0/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Extracting /tmp/mnist/data/train-images-idx3-ubyte.gz
Extracting /tmp/mnist/data/train-labels-idx1-ubyte.gz
Extracting /tmp/mnist/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/mnist/data/t10k-labels-idx1-ubyte.gz
Using mnist-logs/run-2018-09-04T09:52:21 for TensorBoard logs
0 Train accuracy: 0.96 Val accuracy 0.9466
1 Train accuracy: 0.88 Val accuracy 0.9624
2 Train accuracy: 0.92 Val accuracy 0.9704
3 Train accuracy: 0.94 Val accuracy 0.9734
4 Train accuracy: 0.9 Val accuracy 0.9726
5 Train accuracy: 0.88 Val accuracy 0.9762
6 Train accuracy: 0.82 Val accuracy 0.9774
7 Train accuracy: 0.9 Val accuracy 0.9762
8 Train accuracy: 0.9 Val accuracy 0.9768
9 Train accuracy: 0.96 Val accuracy 0.9786


### Transfer learning
Transfer learning refers to re-using a model that was trained for a different task, typically using only the lower, more generic layers. Below, the loading of TensorFlow models is illustrated with examples.

In [2]:
saved_model_meta = SAVED_MODEL_PATH + '.meta'
saver = tf.train.import_meta_graph(SAVED_MODEL_PATH + '.meta')

X = tf.get_default_graph().get_tensor_by_name('X:0')
y = tf.get_default_graph().get_tensor_by_name('y:0')

accuracy = tf.get_default_graph().get_tensor_by_name('eval/accuracy/accuracy:0')
# [n.name for n in tf.get_default_graph().as_graph_def().node]

# for op in tf.get_default_graph().get_operations():
#     print(op.name)

with tf.Session() as sess:
    saver.restore(sess, SAVED_MODEL_PATH)
    acc_test = accuracy.eval(feed_dict={X: mnist.validation.images, y: mnist.validation.labels})
    
print('Accuracy on test set:', acc_test)

INFO:tensorflow:Restoring parameters from mnist-logs/run-2018-09-04T09:52:21/model.ckpt
Accuracy on test set: 0.9786


### Exercise 8
a. Build a neural network with five hidden layers of 100 neurons each, He initialization, and the ELU activation function.

In [3]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
from functools import partial
from datetime import datetime
import os

N_INPUTS = 28*28
N_NEURONS = 100
N_LAYERS = 5
N_CLASSES = 5
N_EPOCHS = 20
BATCH_SIZE = 50
ROOT_LOGDIR = 'chap-11-exercise-8-logs'
LEARNING_RATE = 0.01

tf.reset_default_graph()

def build_placeholders(n_inputs):
    X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
    y = tf.placeholder(tf.int64, shape=(None), name="y")
    return X, y


def build_model(X, y, n_neurons, n_layers, n_classes, learning_rate):
    
    def build_hidden_layers(X, n_layers, n_neurons, init):
        neuron_layer = partial(tf.layers.dense, activation=tf.nn.elu, kernel_initializer=init)
        top_layer = X
        for ind in range(1, n_layers + 1):
            layer_name = 'hidden-layer-' + str(ind)
            top_layer = neuron_layer(top_layer, n_neurons, name=layer_name)
        return top_layer      

    '''
    def build_hidden_layers(X, n_layers, n_neurons, init):
        neuron_layer = partial(tf.layers.dense, activation=tf.nn.elu, kernel_initializer=init)
        hidden1 = neuron_layer(X, n_neurons, name='hidden1')
        hidden2 = neuron_layer(hidden1, n_neurons, name='hidden2')
        hidden3 = neuron_layer(hidden2, n_neurons, name='hidden3')
        hidden4 = neuron_layer(hidden3, n_neurons, name='hidden4')
        hidden5 = neuron_layer(hidden4, n_neurons, name='hidden5')
        return hidden5
    '''

    with tf.name_scope("hidden"):
        he_init = tf.variance_scaling_initializer()
        top_hidden = build_hidden_layers(X, n_layers=5, n_neurons=100, init=he_init)

    
    with tf.name_scope("logits"):
        logits = tf.layers.dense(top_hidden, n_classes, name="logits")


    with tf.name_scope("loss"):
        # This op expects unscaled logits, since it performs a softmax on logits internally for efficiency.
        xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
        loss = tf.reduce_mean(xentropy, name="loss")

    with tf.name_scope("train"):
        optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
        training_op = optimizer.minimize(loss)
    
    # Evaluate performance by checking if the correct label is in top 1:
    with tf.name_scope("eval"):
        correct = tf.nn.in_top_k(logits, y, 1)
        with tf.name_scope("accuracy"):
            accuracy = tf.reduce_mean(tf.cast(correct, tf.float32), name='accuracy')
        
    tf.summary.scalar('accuracy', accuracy)
    summaries = tf.summary.merge_all()
    return { 'accuracy': accuracy, 'summaries': summaries, 'training_op': training_op }


def tb_logdir(root_logdir):   
    now = datetime.now().strftime("%Y-%m-%dT%H:%M:%S")
    return os.path.join(root_logdir, 'run-%s' % now)


def init_feed_dict(mnist, X, y, n_classes):
    
    def extract_data(all_images, all_labels):
        inds_used = mnist.train.labels < n_classes
        images = mnist.train.images[inds_used]
        labels = mnist.train.labels[inds_used]
        return images, labels
    
    def feed_dict(train):
        if train:
            xs, ys = X_train, y_train
        else:
            xs, ys = X_val, y_val
        return {X: xs, y: ys}
    
    train_images, train_labels = extract_data(mnist.train.images, mnist.train.labels)
    val_images, val_labels = extract_data(mnist.validation.images, mnist.validation.labels)
    
    def feed_dict(train):
        if train:
            xs, ys = train_images, train_labels
        else:
            xs, ys = val_images, val_labels

        return {X: xs, y: ys}
    
    return feed_dict


def main():
    X, y = build_placeholders(n_inputs=N_INPUTS)
    model = build_model(X=X, 
                        y=y, 
                        n_neurons=N_NEURONS, 
                        n_layers=N_LAYERS, 
                        n_classes=N_CLASSES,
                        learning_rate=LEARNING_RATE)
    accuracy, summaries, training_op = [model[key] for key in ['accuracy', 'summaries', 'training_op']]

    saver = tf.train.Saver()

    mnist = input_data.read_data_sets("/tmp/mnist/data")

    logdir = tb_logdir(root_logdir=ROOT_LOGDIR)
    print('Using %s for TensorBoard logs' % logdir)

    saved_model_path = os.path.join(logdir, 'model.ckpt')

    with tf.Session() as sess:
        train_writer = tf.summary.FileWriter(logdir + '/train', sess.graph)
        val_writer = tf.summary.FileWriter(logdir + '/val')
        tf.global_variables_initializer().run()
        feed_dict = init_feed_dict(mnist=mnist, X=X, y=y, n_classes=N_CLASSES)
        for epoch in range(N_EPOCHS):
            summary, train_acc, _ = sess.run([summaries, accuracy, training_op], feed_dict=feed_dict(train=True))
            val_summary, val_acc = sess.run([summaries, accuracy], feed_dict=feed_dict(train=False))
            train_writer.add_summary(summary, epoch)
            val_writer.add_summary(val_summary, epoch)
            print('Epoch:', epoch, 'Training acc:', train_acc, 'Validation acc:', val_acc)
        train_writer.close()
        val_writer.close()
        
main()

Extracting /tmp/mnist/data/train-images-idx3-ubyte.gz
Extracting /tmp/mnist/data/train-labels-idx1-ubyte.gz
Extracting /tmp/mnist/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/mnist/data/t10k-labels-idx1-ubyte.gz
Using chap-11-exercise-8-logs/run-2018-09-04T09:52:51 for TensorBoard logs
Epoch: 0 Training acc: 0.18981382 Validation acc: 0.6465154
Epoch: 1 Training acc: 0.6465154 Validation acc: 0.50046366
Epoch: 2 Training acc: 0.50046366 Validation acc: 0.574292
Epoch: 3 Training acc: 0.574292 Validation acc: 0.48163208
Epoch: 4 Training acc: 0.48163208 Validation acc: 0.5812112
Epoch: 5 Training acc: 0.5812112 Validation acc: 0.7403524
Epoch: 6 Training acc: 0.7403524 Validation acc: 0.7392111
Epoch: 7 Training acc: 0.7392111 Validation acc: 0.862187
Epoch: 8 Training acc: 0.862187 Validation acc: 0.9190385
Epoch: 9 Training acc: 0.9190385 Validation acc: 0.92363936
Epoch: 10 Training acc: 0.92363936 Validation acc: 0.90933734
Epoch: 11 Training acc: 0.90933734 Validation acc: 0.9047